<a href="https://colab.research.google.com/github/RutujKhare1/CS6910_Assignment_1/blob/main/FDL_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [2]:
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import wandb

wandb.login(key = '5b3ff6cba361172038b8948f6dace9286a5bbfa0')


In [3]:
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
num_label = 10
num_samples = len(X_train)

In [4]:
print("Train shape : {}, {}".format(X_train.shape, Y_train.shape))
print("Test shape : {}, {}".format(X_test.shape, Y_test.shape))

Train shape : (60000, 28, 28), (60000,)
Test shape : (10000, 28, 28), (10000,)


# Question 1

In [ ]:
num_row = 2
num_col = 5

cnt = 0
images = X_train
labels = Y_train
dic = {0:'T-shirt', 1:'Trowser', 2:'Pullover', 3:'Dress', 4:'Coat', 5:'Sandal', 6:'Shirt', 7:'Sneaker', 8:'Bag', 9:'Ankle Boot'}
temp_labels = []
r_idx = 0
c_idx = 0
fig, axes = plt.subplots(num_row, num_col, figsize=(2*num_col, 2*num_row))
idx = 0
while (cnt<10 and idx < 60000):
  if(labels[idx] not in temp_labels):
    if(c_idx == num_col):
      r_idx = 1
      c_idx = 0
    wandb.init(project = "dl-project_images")
    wandb.log({'Images' : [wandb.Image(images[idx], caption = dic[labels[idx]])]})
    temp_labels.append([labels[idx]])
    ax = axes[r_idx, c_idx]
    ax.imshow(images[idx], cmap='gray')
    ax.set_title('{}'.format(dic[labels[idx]]))
    c_idx += 1
    cnt += 1
  idx += 1 
wandb.finish()
plt.tight_layout()
plt.show()

#Question 2

In [5]:
#Helper functions

def xavierRandom(x, y):
  limit = np.sqrt(6 / float(x + y))
  return np.random.uniform(low=-limit, high=limit, size=(x,y))
  
#activation functions

def sigmoid(x):
  z = x.copy()
  z[x < 0] = np.exp(x[x < 0])/(1 + np.exp(x[x<0]))
  z[x >= 0] = 1/(1+np.exp(-x[x >= 0]))
  return z

def d_sigmoid(x):
  return sigmoid(x)*(1 - sigmoid(x))

def softmax(x):
  e_x = np.exp(x - max(x))
  return e_x / e_x.sum()

def tanh(x):
  return np.tanh(x)
  
def d_tanh(x):
  return 1 - tanh(x)**2

def relu(x):
  return np.maximum(x,0)

def d_relu(x):
  z = x.copy()
  z[x < 0]=0
  z[x > 0]=1
  return z

actFn_dict = {"sigmoid":sigmoid,
               "softmax":softmax,
               "relu":relu,
               "tanh":tanh}
d_actFn_dict = {"sigmoid":d_sigmoid,
               "relu":d_relu,
               "tanh":d_tanh}


In [7]:
class OptimizerHelper:
  def forwardPropagation(self, W, B, X, num_layer, activation):
    H=list()
    A=list()
    a = np.array(W[0]@X.T + B[0])
    h = actFn_dict[activation](a)
    H.append(h)
    A.append(a)
    cur_ip = h

    for i in range(num_layer-1):
      a = np.array(W[i+1]@cur_ip + B[i+1])
      h = actFn_dict[activation](a.T).T
      H.append(h)
      A.append(a)
      cur_ip = h
    a = np.array(W[num_layer]@cur_ip + B[num_layer])
    h = a.T
    A.append(a)
    y = np.array([actFn_dict["softmax"](i) for i in h]).T
    return H, A, y

  def backwardPropagation(self, H, A, y, W, B, X, Y, num_layer, num_label, activation, wt_decay, loss_fn):
    xdim = X.shape[0]
    d_aL = y
    if(loss_fn == "crossEntropy"):
      ey = np.zeros([xdim,num_label], dtype= int)
      for i in range(xdim):
        ey[i][Y[i]] = 1
      d_aL = -(ey.T - y) 
    else:
      ey = np.zeros([xdim,num_label], dtype= int)
      for i in range(xdim):
        ey[i][Y[i]] = 1
      temp = -(ey.T - y)
      d_aL = 2*temp*y*(1-y)

    D_W = list()
    D_B = list()
    for k in reversed(range(1,num_layer+1)):
      d_wL = d_aL@H[k-1].T
      D_W.append(d_wL/xdim)

      d_bL = np.sum(d_aL, axis = 1).reshape(len(d_aL),1)/xdim
      D_B.append(d_bL)
  
      d_hL = W[k].T@d_aL
      dga = d_actFn_dict[activation](A[k-1])
      d_aL = np.multiply(d_hL,dga)

    d_wL = d_aL @ X
    d_bL = d_aL
    d_bL = np.sum(d_bL, axis = 1).reshape(len(d_aL),1)/xdim
    D_W.append(d_wL/xdim)
    D_B.append(d_bL)
    return D_W, D_B

In [8]:
X_mini = np.reshape(X_train, (60000,784))/255.0
num_data_points = 60000
num_labels = 10
num_val_points = num_data_points//10
num_train_points = num_data_points - num_val_points
Xtrain = X_mini[:num_train_points]
Ytrain = Y_train[:num_train_points]
X_valid = X_mini[num_train_points:]
Y_valid =  Y_train[num_train_points:]

#Question 3

In [11]:
class feedforwardNeuralNetwork:
  def __init__(self, num_layers = 3, hidden_size = 64, num_labels = 10, neta = 0.1, epochs = 10, X_train = X_train, Y_train = Y_train, batch = 64, w_init = "xavier", X_val = X_valid, Y_val = Y_valid, activation = 'sigmoid', wt_decay = 0, loss_fn = "crossEntropy"):
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.xdim = X_train.shape[0]
    self.ydim = X_train.shape[1]
    self.num_labels = num_labels
    self.neta = neta
    self.epochs = epochs
    self.X_train = X_train
    self.Y_train = Y_train
    self.X_val = X_val
    self.Y_val = Y_val
    self.batch = batch
    w, b = self.initialiseWeight(w_init)
    self.W = w
    self.B = b
    self.activation = activation
    self.wt_decay = wt_decay
    self.loss_fn = loss_fn
  
  def initialiseWeight(self, method):
    w = list()
    b = list()
    if(method == "xavier"):
      w.append(xavierRandom(self.hidden_size, self.ydim))
      b.append(np.zeros([self.hidden_size,1]))
      for i in range(self.num_layers-1):
        w.append(xavierRandom(self.hidden_size, self.hidden_size))
        b.append(np.zeros([self.hidden_size,1]))
      w.append(xavierRandom(self.num_labels, self.hidden_size))
      b.append(np.zeros([self.num_labels,1]))
    else:
      # w.append(np.random.randn(hidden_size, ydim))
      w.append(np.random.uniform(-1,1, (self.hidden_size, self.ydim)))

      # b.append(np.random.randn(hidden_size,1))
      b.append(np.zeros([self.hidden_size,1]))
      for i in range(self.num_layers-1):
        # w.append(np.random.randn(hidden_size, hidden_size))
        w.append(np.random.uniform(-1,1, (self.hidden_size, self.hidden_size)))
        # b.append(np.random.randn(hidden_size,1))
        b.append(np.zeros([self.hidden_size,1]))
      # w.append(np.random.randn(num_labels, hidden_size))
      w.append(np.random.uniform(-1,1, (self.num_labels, self.hidden_size)))
      # b.append(np.random.randn(num_labels,1))
      b.append(np.zeros([self.num_labels,1])) 
    return w, b

  def calculateCrossEntropy(self, y_hat, Y):
    num_data = y_hat.shape[1]
    Y_one_hot = np.eye(self.num_labels)[Y]
    loss = np.sum(Y_one_hot * np.log(y_hat.T + 1e-15) )
    w_sq = 0
    for i in range(len(self.W)):
      w_sq += np.sum(self.W[i]**2)
    return (-loss + (self.wt_decay*w_sq)/2)/(num_data)

  def calculateMSE(self, y_hat, Y):
    m,n = y_hat.shape
    Y_one_hot = np.eye(self.num_labels)[Y]
    temp = (y_hat.T - Y_one_hot)**2
    loss = np.sum(temp)
    w_sq = 0
    for i in range(len(self.W)):
      w_sq += np.sum(self.W[i]**2)
    return (loss + (self.wt_decay*w_sq)/2)/(n)

  def calculateAccuracy(self, y_hat, Y):
    pred = np.argmax(y_hat, axis=0)
    accuracy = np.mean(pred==Y)
    return accuracy*100

  def feedForward(self, x):
    a = np.array(self.W[0]@x.T + self.B[0])
    h = actFn_dict[self.activation](a.T).T
    cur_ip = h

    for i in range(self.num_layers-1):
      a = np.array(self.W[i+1]@cur_ip + self.B[i+1])
      h = actFn_dict[self.activation](a.T).T
      cur_ip = h
    a = np.array(self.W[self.num_layers]@cur_ip + self.B[self.num_layers])
    h = a.T
    # print(h)
    y = np.array([actFn_dict["softmax"](i) for i in h]).T
    # print(y)
    return y

  

  def momentumGD(self, beta = 0.9, eta = 1.0):
    # print("neta ",self.neta)
    O = OptimizerHelper()
    val_loss = list()
    val_acc = list()
    train_loss = list()
    train_acc = list()
    for k in range(self.epochs):
      for b in range(0, self.xdim, self.batch):
        X_batch = self.X_train[b:b+self.batch]
        Y_batch = self.Y_train[b:b+self.batch]
        H, A, y = O.forwardPropagation(self.W, self.B, X_batch, self.num_layers, self.activation)
        D_W, D_B = O.backwardPropagation(H, A, y, self.W, self.B, X_batch, Y_batch, self.num_layers, self.num_labels, self.activation, self.wt_decay, self.loss_fn)
        D_W = np.flip(D_W)
        D_B = np.flip(D_B)
        if k==0:
          uw = D_W
          ub = D_B
        else:
          uw = beta*prev_uw + eta*D_W
          ub = beta*prev_ub + eta*D_B
        for i in range(self.num_layers+1):
          self.W[i] = self.W[i] - self.neta*uw[i] - self.neta*self.wt_decay*self.W[i]
          self.B[i] = self.B[i] - self.neta*ub[i] - self.neta*self.wt_decay*self.B[i]
        prev_uw = uw
        prev_ub = ub 
      
      y_train_hat = self.feedForward(self.X_train)
      y_val_hat = self.feedForward(self.X_val)
      train_acc.append(self.calculateAccuracy(y_train_hat, self.Y_train))
      val_acc.append(self.calculateAccuracy(y_val_hat, self.Y_val))
      if(self.loss_fn == "crossEntropy"):
        train_loss.append(self.calculateCrossEntropy(y_train_hat, self.Y_train))
        val_loss.append(self.calculateCrossEntropy(y_val_hat, self.Y_val))
      else:
        train_loss.append(self.calculateMSE(y_train_hat, self.Y_train))
        val_loss.append(self.calculateMSE(y_val_hat, self.Y_val))
      print("Epoch {} : Loss:{}\t Accuracy:{}".format(k, train_loss[-1], train_acc[-1]))
      
      
    return train_loss, train_acc, val_loss, val_acc

  def nagGD(self, beta = 0.1, eta = 1.0):
    O = OptimizerHelper()
    val_loss = list()
    val_acc = list()
    train_loss = list()
    train_acc = list()
    for k in range(self.epochs):
      for b in range(0, self.xdim, self.batch):
        X_batch = self.X_train[b:b+self.batch]
        Y_batch = self.Y_train[b:b+self.batch]
        if k==0:
          H, A, y = O.forwardPropagation(self.W, self.B, X_batch, self.num_layers, self.activation)
          D_W, D_B = O.backwardPropagation(H, A, y, self.W, self.B, X_batch, Y_batch, self.num_layers, self.num_labels, self.activation, self.wt_decay, self.loss_fn)
        else:
          u_w = beta*prev_uw
          u_b = beta*prev_ub
          temp_u_w = list()
          temp_u_b = list()
          for i in range(len(self.W)):
            temp_u_w.append(self.W[i]-u_w[i])
            temp_u_b.append(self.B[i]-u_b[i])  
          H, A, y = O.forwardPropagation(temp_u_w, temp_u_b, X_batch, self.num_layers, self.activation)
          D_W, D_B = O.backwardPropagation(H, A, y, temp_u_w, temp_u_b, X_batch, Y_batch, self.num_layers, self.num_labels, self.activation)

        D_W = np.flip(D_W)
        D_B = np.flip(D_B)
        if k==0:
          uw = D_W
          ub = D_B
        else:
          uw = beta*prev_uw + eta*D_W
          ub = beta*prev_ub + eta*D_B
        for i in range(self.num_layers+1):
          self.W[i] = self.W[i] - self.neta*uw[i] - self.neta*self.wt_decay*self.W[i]
          self.B[i] = self.B[i] - self.neta*ub[i] - self.neta*self.wt_decay*self.B[i]
        prev_uw = uw
        prev_ub = ub 
      
      y_train_hat = self.feedForward(self.X_train)
      y_val_hat = self.feedForward(self.X_val)
      train_acc.append(self.calculateAccuracy(y_train_hat, self.Y_train))
      val_acc.append(self.calculateAccuracy(y_val_hat, self.Y_val))
      if(self.loss_fn == "crossEntropy"):
        train_loss.append(self.calculateCrossEntropy(y_train_hat, self.Y_train))
        val_loss.append(self.calculateCrossEntropy(y_val_hat, self.Y_val))
      else:
        train_loss.append(self.calculateMSE(y_train_hat, self.Y_train))
        val_loss.append(self.calculateMSE(y_val_hat, self.Y_val))
    return train_loss, train_acc, val_loss, val_acc
    
  
  def stochasticGD(self):
    O = OptimizerHelper()
    val_loss = list()
    val_acc = list()
    train_loss = list()
    train_acc = list()
    for k in range(self.epochs):
      for b in range(0, self.xdim, self.batch):
        X_batch = self.X_train[b:b+self.batch]
        Y_batch = self.Y_train[b:b+self.batch]
        H, A, y = O.forwardPropagation(self.W, self.B, X_batch, self.num_layers, self.activation)
        D_W, D_B = O.backwardPropagation(H, A, y, self.W, self.B, X_batch, Y_batch, self.num_layers, self.num_labels, self.activation, self.wt_decay, self.loss_fn)
        D_W = np.flip(D_W)
        D_B = np.flip(D_B)
        # print(self.B[0].shape)
        # print(D_B[0].shape)
        for i in range(self.num_layers+1):
          self.W[i] = self.W[i] - self.neta*D_W[i] - self.neta*self.wt_decay*self.W[i]
          self.B[i] = self.B[i] - self.neta*D_B[i] - self.neta*self.wt_decay*self.B[i]
      
      y_train_hat = self.feedForward(self.X_train)
      y_val_hat = self.feedForward(self.X_val)
      train_acc.append(self.calculateAccuracy(y_train_hat, self.Y_train))
      val_acc.append(self.calculateAccuracy(y_val_hat, self.Y_val))
      if(self.loss_fn == "crossEntropy"):
        train_loss.append(self.calculateCrossEntropy(y_train_hat, self.Y_train))
        val_loss.append(self.calculateCrossEntropy(y_val_hat, self.Y_val))
      else:
        train_loss.append(self.calculateMSE(y_train_hat, self.Y_train))
        val_loss.append(self.calculateMSE(y_val_hat, self.Y_val))
    return train_loss, train_acc, val_loss, val_acc

  def RMSprop(self, beta = 0.9, eps = 1e-8):
    # print("neta ",self.neta)
    O = OptimizerHelper()
    val_loss = list()
    val_acc = list()
    train_loss = list()
    train_acc = list()
    for k in range(self.epochs):
      for b in range(0, self.xdim, self.batch):
        X_batch = self.X_train[b:b+self.batch]
        Y_batch = self.Y_train[b:b+self.batch]
        H, A, y = O.forwardPropagation(self.W, self.B, X_batch, self.num_layers, self.activation)
        D_W, D_B = O.backwardPropagation(H, A, y, self.W, self.B, X_batch, Y_batch, self.num_layers, self.num_labels, self.activation, self.wt_decay, self.loss_fn)
        D_W = np.flip(D_W)
        D_B = np.flip(D_B)
        if k==0:
          uw = (1-beta)*np.square(D_W)
          ub = (1-beta)*np.square(D_B)
        else:
          uw = beta*prev_uw + (1-beta)*np.square(D_W)
          ub = beta*prev_ub + (1-beta)*np.square(D_B)
        for i in range(self.num_layers+1):
          self.W[i] = self.W[i] - self.neta*D_W[i]/(np.sqrt(uw[i]) + eps) - self.neta*self.wt_decay*self.W[i]
          self.B[i] = self.B[i] - self.neta*D_B[i]/(np.sqrt(ub[i]) + eps) - self.neta*self.wt_decay*self.B[i]
        prev_uw = uw
        prev_ub = ub 
      
      y_train_hat = self.feedForward(self.X_train)
      y_val_hat = self.feedForward(self.X_val)
      train_acc.append(self.calculateAccuracy(y_train_hat, self.Y_train))
      val_acc.append(self.calculateAccuracy(y_val_hat, self.Y_val))
      if(self.loss_fn == "crossEntropy"):
        train_loss.append(self.calculateCrossEntropy(y_train_hat, self.Y_train))
        val_loss.append(self.calculateCrossEntropy(y_val_hat, self.Y_val))
      else:
        train_loss.append(self.calculateMSE(y_train_hat, self.Y_train))
        val_loss.append(self.calculateMSE(y_val_hat, self.Y_val))
    return train_loss, train_acc, val_loss, val_acc

  def adam(self, beta1 = 0.9, beta2 = 0.999, eps = 1e-8):
    # print("neta ",self.neta)
    O = OptimizerHelper()
    val_loss = list()
    val_acc = list()
    train_loss = list()
    train_acc = list()
    for k in range(self.epochs):
      for b in range(0, self.xdim, self.batch):
        X_batch = self.X_train[b:b+self.batch]
        Y_batch = self.Y_train[b:b+self.batch]
        H, A, y = O.forwardPropagation(self.W, self.B, X_batch, self.num_layers, self.activation)
        D_W, D_B = O.backwardPropagation(H, A, y, self.W, self.B, X_batch, Y_batch, self.num_layers, self.num_labels, self.activation, self.wt_decay, self.loss_fn)
        D_W = np.flip(D_W)
        D_B = np.flip(D_B)
        if k==0:
          mw = (1-beta1)*D_W
          mb = (1-beta1)*D_B
          uw = (1-beta2)*np.square(D_W)
          ub = (1-beta2)*np.square(D_B)
        else:
          mw = beta1*mw + (1-beta1)*D_W
          mb = beta1*mb + (1-beta1)*D_B
          uw = beta2*uw + (1-beta2)*np.square(D_W)
          ub = beta2*ub + (1-beta2)*np.square(D_B)

        mw_hat = mw / (1-np.power(beta1,k+1))
        mb_hat = mb / (1-np.power(beta1,k+1))
        uw_hat = uw / (1-np.power(beta2,k+1))
        ub_hat = ub / (1-np.power(beta2,k+1))
        for i in range(self.num_layers+1):
          self.W[i] = self.W[i] - self.neta*mw_hat[i]/(np.sqrt(uw_hat[i]) + eps) - self.neta*self.wt_decay*self.W[i]
          self.B[i] = self.B[i] - self.neta*mb_hat[i]/(np.sqrt(ub_hat[i]) + eps) - self.neta*self.wt_decay*self.B[i]
      
      y_train_hat = self.feedForward(self.X_train)
      y_val_hat = self.feedForward(self.X_val)
      train_acc.append(self.calculateAccuracy(y_train_hat, self.Y_train))
      val_acc.append(self.calculateAccuracy(y_val_hat, self.Y_val))
      if(self.loss_fn == "crossEntropy"):
        train_loss.append(self.calculateCrossEntropy(y_train_hat, self.Y_train))
        val_loss.append(self.calculateCrossEntropy(y_val_hat, self.Y_val))
      else:
        train_loss.append(self.calculateMSE(y_train_hat, self.Y_train))
        val_loss.append(self.calculateMSE(y_val_hat, self.Y_val))
      print("Epoch {} : Loss:{}\t Accuracy:{}".format(k, train_loss[-1], train_acc[-1]))
    return train_loss, train_acc, val_loss, val_acc

  def nAdam(self, beta1 = 0.9, beta2 = 0.999, eps = 1e-8):
    # print("neta ",self.neta)
    val_loss = list()
    val_acc = list()
    train_loss = list()
    train_acc = list()
    O = OptimizerHelper()
    for k in range(self.epochs):
      for b in range(0, self.xdim, self.batch):
        X_batch = self.X_train[b:b+self.batch]
        Y_batch = self.Y_train[b:b+self.batch]
        H, A, y = O.forwardPropagation(self.W, self.B, X_batch, self.num_layers, self.activation)
        D_W, D_B = O.backwardPropagation(H, A, y, self.W, self.B, X_batch, Y_batch, self.num_layers, self.num_labels, self.activation, self.wt_decay, self.loss_fn)
        D_W = np.flip(D_W)
        D_B = np.flip(D_B)
        if k==0:
          mw = (1-beta1)*D_W
          mb = (1-beta1)*D_B
          uw = (1-beta2)*np.square(D_W)
          ub = (1-beta2)*np.square(D_B)
        else:
          mw = beta1*mw + (1-beta1)*D_W
          mb = beta1*mb + (1-beta1)*D_B
          uw = beta2*uw + (1-beta2)*np.square(D_W)
          ub = beta2*ub + (1-beta2)*np.square(D_B)

        mw_hat = mw / (1-np.power(beta1,k+1))
        mb_hat = mb / (1-np.power(beta1,k+1))
        uw_hat = uw / (1-np.power(beta2,k+1))
        ub_hat = ub / (1-np.power(beta2,k+1))
        for i in range(self.num_layers+1):
          self.W[i] = self.W[i] - self.neta/(np.sqrt(uw_hat[i]) + eps)*(beta1*mw_hat[i] + (1-beta1)*D_W[i] / (1-np.power(beta1,k+1))) - self.neta*self.wt_decay*self.W[i]
          self.B[i] = self.B[i] - self.neta/(np.sqrt(ub_hat[i]) + eps)*(beta1*mb_hat[i] + (1-beta1)*D_B[i] / (1-np.power(beta1,k+1))) - self.neta*self.wt_decay*self.B[i] 
      
      y_train_hat = self.feedForward(self.X_train)
      y_val_hat = self.feedForward(self.X_val)
      train_acc.append(self.calculateAccuracy(y_train_hat, self.Y_train))
      val_acc.append(self.calculateAccuracy(y_val_hat, self.Y_val))
      if(self.loss_fn == "crossEntropy"):
        train_loss.append(self.calculateCrossEntropy(y_train_hat, self.Y_train))
        val_loss.append(self.calculateCrossEntropy(y_val_hat, self.Y_val))
      else:
        # print("---------MSE-------------")
        train_loss.append(self.calculateMSE(y_train_hat, self.Y_train))
        val_loss.append(self.calculateMSE(y_val_hat, self.Y_val))
      # print("Epoch {} : Loss:{}\t Accuracy:{}".format(k, train_loss[-1], train_acc[-1]))
    return train_loss, train_acc, val_loss, val_acc


  def accuracy(self, Y):
    pred = np.argmax(Y, axis=1)
    print(pred.shape)

#Question 4

In [71]:
def train():
  wandb.init()
  num_layers = wandb.config.num_hidden_layer
  hidden_size = wandb.config.hidden_size
  activation = wandb.config.activation
  epochs = wandb.config.epochs
  optimizer_name = wandb.config.optimizer
  neta = wandb.config.lr
  batch = wandb.config.batch_size
  w_init = wandb.config.weight_init
  wt_decay = wandb.config.wt_decay
  loss_fn = wandb.config.loss_fn

  wandb.run.name = f'hln_{num_layers}_hls_{hidden_size}_op_{optimizer_name}_hla_{activation}_lr_{neta}_ep_{epochs}_bs_{batch}_winit_{w_init}_wt_decay_{wt_decay}'

  F = feedforwardNeuralNetwork(neta = neta, hidden_size = hidden_size, num_layers = num_layers, X_train = Xtrain, Y_train = Ytrain, X_val = X_valid, Y_val = Y_valid, activation = activation, batch = batch, epochs = epochs, w_init = w_init, wt_decay = wt_decay, loss_fn = loss_fn)
  if(optimizer_name == "sgd"):
    train_loss, train_acc, val_loss, val_acc = F.stochasticGD()
  elif (optimizer_name == "momentum"):
    train_loss, train_acc, val_loss, val_acc = F.momentumGD()
  elif (optimizer_name == "nesterov"):
    train_loss, train_acc, val_loss, val_acc = F.nagGD()
  elif (optimizer_name == "rmsprop"):
    train_loss, train_acc, val_loss, val_acc = F.RMSprop()
  elif (optimizer_name == "adam"):
    train_loss, train_acc, val_loss, val_acc = F.adam()
  elif (optimizer_name == "nadam"):
    train_loss, train_acc, val_loss, val_acc = F.nAdam()

  for i in range(len(train_loss)):
    wandb.log({'training_loss': train_loss[i],
              'training_accuracy': train_acc[i],
              'validation_loss': val_loss[i],
              'validation_accuracy': val_acc[i]
              })

In [ ]:
sweep_configuration = {
    'method': 'random',
    'name': 'dl-project',
    'metric': {
        'goal': 'minimize', 
        'name': 'validation_loss'
        },
    'parameters': {
        'batch_size': {'values': [16, 32, 64]},
        'epochs': {'values': [10, 15]},
        'num_hidden_layer':{'values' : [3, 4, 5]},
        'hidden_size': {'values' : [32, 64, 128]},
        'lr': {'values' : [1e-2, 1e-3, 1e-4]},
        'optimizer':{'values' : ['sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam']},
        'weight_init':{'values' : ['random','xavier']},
        'activation' : {'values' : ['sigmoid','tanh','relu']},
        'wt_decay': {'values' : [0, 0.0001, 0.001]},
        'loss_fn': {'values' : ['crossEntropy']}
     }
}




sweep_id = wandb.sweep(sweep=sweep_configuration, project='dl-project_FINALV2')
wandb.agent(sweep_id, function=train)
wandb.finish()

#Question 7


In [ ]:
#Best Hyper-parameter
F = feedforwardNeuralNetwork(neta = 0.001, hidden_size = 128, num_layers = 4, X_train = Xtrain, Y_train = Ytrain, X_val = X_valid, Y_val = Y_valid, activation="tanh", epochs = 15, wt_decay = 0.0, batch = 64)
train_loss, train_acc, val_loss, val_acc = F.adam()
print(train_loss)
print(train_acc)

Xtest = np.reshape(X_test, (len(X_test),784))/255
y_hat = F.feedForward(Xtest)
print("Test Accuracy : {}".format(F.calculateAccuracy(y_hat, Y_test)))

In [ ]:
def plotConfusionMatrixUpd(y_true, y_pred):
  labels = [i for i in range(num_labels)]
  cm = np.zeros((len(labels), len(labels)), dtype=np.int32)
  for i in range(len(y_true)):
      true_class_index = np.where(labels == y_true[i])[0][0]
      pred_class_index = np.where(labels == y_pred[i])[0][0]
      cm[true_class_index][pred_class_index] += 1
  cm = np.flip(cm,0)
  cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

  fig, ax = plt.subplots()
  im = ax.imshow(cm_normalized, cmap=plt.cm.Blues, interpolation='nearest')
  cbar = ax.figure.colorbar(im, ax=ax)
  ax.set_xticks(labels)
  ax.set_yticks(labels)
  ax.set_xticklabels(labels)
  ax.set_yticklabels(labels[::-1])
  plt.setp(ax.get_xticklabels(), ha="right",
            rotation_mode="anchor")
  for i in range(len(labels)):
      for j in range(len(labels)):
          text = ax.text(j, i, format(cm[i, j], 'd'),
                          ha="center", va="center", color="silver")

  ax.set_xlabel('Predicted labels')
  ax.set_ylabel('True labels')
  ax.set_title('Confusion matrix')
  # wandb.init(project='dl-assignment-cnfmatFN')
  # wandb.log({'Confusion Matrix' : [wandb.Image(im)]})
  # wandb.finish()
  plt.show()

In [ ]:
Xtest = np.reshape(X_test, (len(X_test),784))/255
y_pred = F.feedForward(Xtest)
y_test_pred = np.argmax(y_pred, axis=0)
plotConfusionMatrixUpd(y_test_pred, Y_test)

#Question 8 

In [74]:
best_sweep_configuration = {
    'method': 'random',
    'name': 'dl-project',
    'metric': {
        'goal': 'minimize', 
        'name': 'validation_loss'
        },
    'parameters': {
        'batch_size': {'values': [64]},
        'epochs': {'values': [15]},
        'num_hidden_layer':{'values' : [4]},
        'hidden_size': {'values' : [128]},
        'lr': {'values' : [1e-3]},
        'optimizer':{'values' : ['adam']},
        'weight_init':{'values' : ['xavier']},
        'activation' : {'values' : ['tanh']},
        'wt_decay': {'values' : [0.0001]},
        'loss_fn': {'values' : ['crossEntropy', 'MSE']}
     }
}

In [ ]:
sweep_id = wandb.sweep(sweep=best_sweep_configuration, project='dl-project_CE_MSE2')
wandb.agent(sweep_id, function=train, count = 2)
wandb.finish()

In [ ]:
# F = feedforwardNeuralNetwork(neta = 0.001, hidden_size = 32, num_layers = 3, X_train = Xtrain, Y_train = Ytrain, X_val = X_valid, Y_val = Y_valid, activation="relu", batch = 32, epochs = 5, wt_decay = 0.005)
# train_loss, train_acc, val_loss, val_acc = F.momentumGD()
# print(train_loss)
# print(train_acc)

In [ ]:
# F = feedforwardNeuralNetwork(neta = 0.001, hidden_size = 32, num_layers = 3, X_train = Xtrain, Y_train = Ytrain, X_val = X_valid, Y_val = Y_valid, activation="relu", batch = 32, epochs = 5, wt_decay = 0.005, loss_fn = "mse")
# train_loss, train_acc, val_loss, val_acc = F.momentumGD()
# print(train_loss)
# print(train_acc)

In [ ]:
# F = feedforwardNeuralNetwork(neta = 0.05, hidden_size = 64, num_layers = 3, X_train = Xtrain, Y_train = Ytrain, X_val = X_valid, Y_val = Y_valid, activation="tanh")
# train_loss, train_acc, val_loss, val_acc = F.stochasticGD()
# print(train_loss)
# print(train_acc)

In [ ]:
# F = feedforwardNeuralNetwork(neta = 0.05, hidden_size = 64, num_layers = 3, X_train = Xtrain, Y_train = Ytrain, X_val = X_valid, Y_val = Y_valid, activation="tanh")
# train_loss, train_acc, val_loss, val_acc = F.nagGD()
# print(train_loss)
# print(train_acc)

In [ ]:
# F = feedforwardNeuralNetwork(neta = 0.0001, hidden_size = 32, num_layers = 3, X_train = Xtrain, Y_train = Ytrain, X_val = X_valid, Y_val = Y_valid, activation="relu")
# train_loss, train_acc, val_loss, val_acc = F.RMSprop()
# print(train_loss)
# print(train_acc)

In [ ]:
# F = feedforwardNeuralNetwork(neta = 0.05, hidden_size = 32, num_layers = 3, X_train = Xtrain, Y_train = Ytrain, X_val = X_valid, Y_val = Y_valid, activation="sigmoid", wt_decay = 0.0)
# train_loss, train_acc, val_loss, val_acc = F.adam()
# print(train_loss)
# print(train_acc)

In [ ]:
# F = feedforwardNeuralNetwork(neta = 0.001, hidden_size = 64, num_layers = 3, X_train = Xtrain, Y_train = Ytrain, X_val = X_valid, Y_val = Y_valid, activation="tanh", epochs = 10, wt_decay = 0.0001)
# train_loss, train_acc, val_loss, val_acc = F.nAdam()
# print(train_loss)
# print(train_acc)

In [ ]:
# F = feedforwardNeuralNetwork(neta = 0.001, hidden_size = 128, num_layers = 4, X_train = Xtrain, Y_train = Ytrain, X_val = X_valid, Y_val = Y_valid, activation="tanh", epochs = 15, wt_decay = 0.0, batch = 64, loss_fn = "MSE")
# train_loss, train_acc, val_loss, val_acc = F.adam()
# print(train_loss)
# print(train_acc)
# #best hyper parameter MSE

In [ ]:
# X_mini = np.reshape(X_train, (60000,784))/255
# num_data_points = 32000
# X_mini = X_mini[:num_data_points]
# Y_mini = Y_train[:num_data_points]
# F = feedforwardNeuralNetwork(neta = 0.05, hidden_size = 32, num_layers = 3, X_train = X_mini, Y_train = Y_mini)
# F.nagGD()

# y_hat = F.feedForward(X_mini)
# pred = np.argmax(y_hat, axis=0)
# # print(Y_test[:5])
# # print(pred)
# accuracy = np.mean(pred==Y_mini)
# print("Train Accuracy : {}".format(accuracy))

# Xtest = np.reshape(X_test, (10000,784))/255
# y_hat = F.feedForward(Xtest)
# pred = np.argmax(y_hat, axis=0)
# # print(Y)
# # print(pred)
# accuracy = np.mean(pred==Y_test)
# print("Test Accuracy : {}".format(accuracy))

In [15]:
!python /content/drive/MyDrive/DLA1/train.py

2023-03-19 14:43:46.340023: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-19 14:43:46.340159: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-19 14:43:46.340183: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/usr/local/lib/python3.9/dist-packages/numpy/lib/function_base.py:334: VisibleDeprecationWarning: Creating an n